In [ ]:
%run -i ../python/common.py
%run -i ../python/ln_preamble.py
# Make three extra terminals to serve as example of 3 terminal emulators
# We do this to avoid having to change the dimensions of our standard
# Editor, Build and debugger terminals
try:
    EXTERM1=TERMINALS[3]['name']
except IndexError:
    EXTERM1=mkTerm()
try:
    EXTERM2=TERMINALS[4]['name']
except IndexError:
    EXTERM2=mkTerm()
try:
    EXTERM3=TERMINALS[5]['name']
except IndexError:
    EXTERM3=mkTerm()
try:
    HHTERM=TERMINALS[6]['name']
except IndexError:
    HHTERM=mkTerm()

# SLS Lecture 2 : Part II

In [ ]:
print("hello")

#### PATHS and the ROOT
<img align="left" src="../images/dirtree.png" style="background-color:white;" width="40%">  

`/` $+$ `home` $+$ `jonathan` $+$ `Classes` $+$ `CS` $+$ `210` $+$ `Assignment 1` $+$ `Problem 1`     

`/` $+$ `home` $+$ `jonathan` $+$ `Classes` $+$ `CS` $+$ `210` $+$ `Assignment 2` $+$ `Problem 1` 

`/` $+$ `home` $+$ `jonathan` $+$ `Classes` $+$ `CS` $+$ `210` $+$ `Assignment 3` $+$ `Problem 1` 

"Full" path name: join independent components with the "/" character.  So the above three files as proper unix path names would be:

See. https://www.gnu.org/software/bash/manual/bash.html#Shell-Commands
2. Breakdown complex commands into simple commands
  - complex commands are a composition of simple commands:
      - pipelines 
        - connecting inputs and outputs of commands together using `|`
        - `ls -1 /bin | wc -l`
      - lists 
        - one or more pipelines separated by one of: `;`, `&&`, `||` or `&` and terminated by `;` or `;` or `&` or a newline.
          - `echo "about to run ls"; ls; echo "done running ls"`
          - `cmd1 && cmd2` : run `cmd2` if `cmd1` returns "success"
          - `cmd1 || cmd2` : run `cmd2` if `cmd1` returns "not success" aka "failure"
          - `cmd1 & cmd2 ` :  run `cmd1` asynchronously in the background and run `cmd2` "normally"
             - lots more to say about asynchronous commands
      - compound commands: Shell programming constructs (these can span lines)
        - loops: `until`, `while`, `for`, `do`, `done`, `break`, `continue`
          - there are two type of `for` loops
        - conditionals: `if`, `then`, `elif`, `else`, `fi`, `case`, `select`, `(())`, `[[]]`
        - grouping: `()`, `{}`
  - functions
 

In [ ]:
!tput

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/shlldirfile.sh", 
    lang="bash", 
    title='# File and Directories - Part I',
    w="100%",
    h="500px")))

In [ ]:
display(showET(title=""))